In [ ]:
%matplotlib inline
import os
import re
import sys
from oqmbt.oqt_project import OQtProject
from openquake.hazardlib.tom import PoissonTOM
from openquake.hazardlib.source import SimpleFaultSource
from openquake.hazardlib.const import TRT
from openquake.hazardlib.scalerel.wc1994 import WC1994

In [ ]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
try:
    source_ids_list = getattr(oqtkp,'active_source_id')
except:
    print('Active source ID not defined in the OQMBT project')
    source_ids_list = ['sf1']
print('Processing fault source with ID:', source_ids_list)

## Get the source

In [ ]:
assert len(source_ids_list) == 1
src = model.sources[source_ids_list[0]]

## Creating the nrml source

In [ ]:
rupture_mesh_spacing = float(model.fault_rupture_mesh_spacing)
rupt_aspect_ratio = float(model.fault_rupt_aspect_ratio)
temporal_occurrence_model = PoissonTOM(1.0)

if hasattr(src, 'tectonic_region_type'):
    if src.tectonic_region_type == 'Active Shallow Crust':
        tectonic_region_type = TRT.ACTIVE_SHALLOW_CRUST
    elif src.tectonic_region_type == 'Stable Shallow Crust':
        tectonic_region_type = TRT.STABLE_CONTINENTAL
    else:
        raise ValueError('Unknown tectonic region')
else:
    tectonic_region_type = TRT.ACTIVE_SHALLOW_CRUST
    
flag = False

if hasattr(src, 'mfd') and src.mfd is not None and len(src.mfd.occurrence_rates):
    
    if 'scalerel' in src.__dict__:
        scalerel = src.scalerel
    else:
        scalerel = WC1994()
    
    nsrc = SimpleFaultSource(
            source_id = src.source_id, 
            name = src.name, 
            tectonic_region_type = tectonic_region_type, 
            mfd = src.mfd, 
            rupture_mesh_spacing = float(rupture_mesh_spacing), 
            magnitude_scaling_relationship = scalerel, 
            rupture_aspect_ratio = rupt_aspect_ratio, 
            temporal_occurrence_model = temporal_occurrence_model, 
            upper_seismogenic_depth = float(src.upper_seismogenic_depth), 
            lower_seismogenic_depth = float(src.lower_seismogenic_depth), 
            fault_trace = src.trace,
            dip = src.dip,
            rake = src.rake)
    flag = True
else:
    print('skipping: ', src.source_id)

## Update the project

In [ ]:
if flag:
    if hasattr(model, 'nrml_sources'):
        model.nrml_sources[src.source_id] = nsrc
    else:
        model.nrml_sources = {src.source_id : nsrc}
    oqtkp.save()
    print ('Source id                       : %s' % (src.source_id))
    print ('Added to nrml sources in model  : %s' % (model_id))